In [6]:
import cv2
import os
from tqdm import tqdm
import glob

In [7]:
def draw_yolo_boxes(image_path, label_path, save_path=None):
    """
    Draw YOLO bounding boxes on an image.

    image_path: path to image
    label_path: path to YOLO label .txt file
    save_path: optional, path to save the output image
    """

    # Load the image
    img = cv2.imread(image_path)
    if img is None:
        print(f"Cannot load image: {image_path}")
        return

    h, w, _ = img.shape

    # Read YOLO labels
    with open(label_path, "r") as f:
        lines = f.readlines()

    for line in lines:
        cat, x_c, y_c, bw, bh = map(float, line.split())

        # Convert from YOLO to pixel coords
        x_center = x_c * w
        y_center = y_c * h
        box_width = bw * w
        box_height = bh * h

        x1 = int(x_center - box_width / 2)
        y1 = int(y_center - box_height / 2)
        x2 = int(x_center + box_width / 2)
        y2 = int(y_center + box_height / 2)

        color = (0,255,0) #class 1
        if cat == 0: color = (0,0,255)
        elif cat == 2: color = (255,0,0)

        # Draw bounding box
        cv2.rectangle(img, (x1, y1), (x2, y2), color, 2)
        cv2.putText(img, f"class {int(cat)}", (x1, y1 - 5),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

    # Save or show
    if save_path:
        cv2.imwrite(save_path, img)
        # print(f"Saved preview → {save_path}")
    else:
        cv2.imshow("YOLO Bounding Boxes", img)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

In [8]:
image_dir = "../../data/annotated_pictures/combined/images/"
label_dir = "../../data/annotated_pictures/combined/labels/"
output_dir = "combined_preview/"

In [9]:
os.makedirs(output_dir, exist_ok=True)
files = glob.glob(os.path.join(output_dir, "*"))

for f in files:
    try:
        os.remove(f)
    except IsADirectoryError:
        print(f"Skipped directory: {f}")
    except Exception as e:
        print(f"Error deleting {f}: {e}")


In [10]:
for file in tqdm(os.listdir(image_dir)):
    if file.endswith((".jpg", ".png", ".jpeg")):
        img_path = os.path.join(image_dir, file)
        txt_path = os.path.join(label_dir, file.replace(".jpg", ".txt")
                                            .replace(".png", ".txt")
                                            .replace(".jpeg", ".txt"))

        if os.path.exists(txt_path):
            save_path = os.path.join(output_dir, file)
            draw_yolo_boxes(img_path, txt_path, save_path)
        else:
            print(f"No label found for {file}")

100%|██████████| 451/451 [00:59<00:00,  7.62it/s]


In [11]:
# rename 
import os
import re

already_ok_pattern = re.compile(r"^\d+$")

for filename in tqdm(os.listdir(output_dir), desc='Renaming files'):
    name, ext = os.path.splitext(filename)
    if os.path.isdir(os.path.join(output_dir, filename)):
        continue

    if already_ok_pattern.match(name):
        continue
    if "-" not in name:
        continue

    order_number = name.split("-")[-1]
    if not order_number.isdigit():
        continue

    new_name = order_number + ext
    old_path = os.path.join(output_dir, filename)
    new_path = os.path.join(output_dir, new_name)

    if os.path.exists(new_path):
        print(f"Skipping {filename}: target {new_name} already exists.")
        continue

    os.rename(old_path, new_path)


Renaming files: 100%|██████████| 451/451 [00:00<00:00, 3689.63it/s]

Skipping 70d42ac8-498.png: target 498.png already exists.
Skipping 7529784d-390.png: target 390.png already exists.
